In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
from actor_net import ActorNet
import numpy as np
from gameworlds.hex_world import HexState

Config file:  7x7zero.json


In [3]:
with open('data/2022-03-28T18-29-14_7x7/2022-03-28T18-29-14_7x7.npy', 'rb') as f: # First VM 
    states = np.load(f)
    y_train = np.load(f)
    y_train_value = np.load(f)

In [4]:
with open('data/2022-03-28T20-40-02_7x7/2022-03-28T20-40-02_7x7.npy', 'rb') as f: # Second VM
    states2 = np.load(f)
    y_train2 = np.load(f)
    y_train_value2 = np.load(f)

In [64]:
with open('data/2022-03-29T15-15-41_7x7/2022-03-29T15-15-41_7x7.npy', 'rb') as f: # 6 paralell , too high C_puct
    states3 = np.load(f)
    y_train3 = np.load(f)
    y_train_value3 = np.load(f)

In [58]:
with open('data/2022-03-28T20-38-02_7x7/2022-03-28T20-38-02_7x7.npy', 'rb') as f: # Third VM
    states4 = np.load(f)
    y_train4 = np.load(f)
    y_train_value4 = np.load(f)

In [61]:
with open('data/2022-03-28T20-41-03_7x7/2022-03-28T20-41-03_7x7.npy', 'rb') as f: # Fourth VM
    states5 = np.load(f)
    y_train5 = np.load(f)
    y_train_value5 = np.load(f)

In [146]:
with open('data/2022-03-27T19-41-30_7x7\dataset/2022-03-27T19-41-30_7x7.npy', 'rb') as f: # Fourth VM
    states6 = np.load(f)
    y_train6 = np.load(f)
    y_train_value6 = np.load(f)

In [9]:
with open('data/2022-03-28T19-37-32_7x7/2022-03-28T19-37-32_7x7.npy', 'rb') as f: # 500 ep
    states7 = np.load(f)
    y_train7 = np.load(f)
    y_train_value7 = np.load(f)

In [3]:
with open('data/2022-03-28T20-35-56_7x7/2022-03-28T20-35-56_7x7.npy', 'rb') as f: # 0.005 ep
    states8 = np.load(f)
    y_train8 = np.load(f)
    y_train_value8 = np.load(f)

In [12]:
with open('data/middle40/2022-03-30T01-29-19_7x7.npy', 'rb') as f: # 500 ep
    states0 = np.load(f)
    y_train0 = np.load(f)
    y_train_value0 = np.load(f)

In [13]:
states0.shape

(29320, 50)

In [15]:
np.unique(states0[-20000:], axis=0).shape

(8398, 50)

In [147]:
states6, y_train6, y_train_value6 = deduplicate(states6, y_train6, y_train_value6)

In [134]:
states6.shape

(25716, 50)

In [125]:
x = np.concatenate((states, states2, states3, states4, states5))
y = np.concatenate((y_train, y_train2, y_train3, y_train4, y_train5))
y_val = np.concatenate((y_train_value, y_train_value2,  y_train_value3,  y_train_value4,  y_train_value5))

In [127]:
x.shape

(92324, 50)

In [111]:
def deduplicate(states, y, y_v):
    unique = np.unique(states, axis=0)
    new_state = []
    new_y = []
    new_y_v =[]
    for s in unique:

        ind = np.all(states == s, axis=1)

        avg_y = np.average(y[ind], axis=0)
        avg_v = np.average(y_v[ind], axis=0)

        new_state.append(s)
        new_y.append(avg_y)
        new_y_v.append(avg_v)

    return np.array(new_state), np.array(new_y), np.array(new_y_v)



In [126]:
x_unique, y_unique, v_unique = deduplicate(states=x, y=y, y_v=y_val)

In [128]:
x_unique.shape

(28104, 50)

In [149]:
super_x, super_y, super_v = deduplicate(np.concatenate((x_unique, states6)), np.concatenate((y_unique, y_train6)), np.concatenate((v_unique,y_train_value6)))

In [151]:
np.unique(super_x, axis=0).shape

(53724, 50)

In [157]:
with open(f"data/super.npy", "wb") as f:
    np.save(f, super_x)
    np.save(f, super_y)
    np.save(f, super_v)

Config file:  7x7zero.json


In [2]:
with open(f"data/super.npy", "rb") as f:
    super_x = np.load(f)
    super_y = np.load(f)
    super_v = np.load(f)

In [40]:
n_last = 5000

In [41]:
x = np.concatenate((states[-n_last:], states2[-n_last:]))
y = np.concatenate((y_train[-n_last:], y_train2[-n_last:]))
y_val = np.concatenate((y_train_value[-n_last:], y_train_value2[-n_last:]))

In [16]:

x_train = np.array(list(map(HexState.from_array_to_vector, states0)))

In [10]:
with open(f"data/super_x.npy", "wb") as f:
    np.save(f,x_train)

In [3]:
with open(f"data/super_x.npy", "rb") as f:
    x_train = np.load(f)

In [8]:
anet = ActorNet("./data/middle40")

In [9]:
anet.load_params(99, "7x7")

In [5]:
scaled = super_y ** 5

In [6]:
scaled = scaled / np.sum(scaled, axis=1).reshape(len(scaled), 1)

In [17]:
anet.train(x_train=x_train[10000:],y_train=y_train0[10000:], y_train_value=y_train_value0[10000:], epochs=3, batch_size=64)

Epoch 1/3
302/302 [==============================] - 6s 21ms/step - loss: 1.5444 - policy_loss: 1.3901 - value_loss: 0.1543 - policy_accuracy: 0.6931 - value_accuracy: 0.0468
Epoch 2/3
302/302 [==============================] - 6s 20ms/step - loss: 1.3374 - policy_loss: 1.2249 - value_loss: 0.1125 - policy_accuracy: 0.7371 - value_accuracy: 0.0477
Epoch 3/3
302/302 [==============================] - 6s 20ms/step - loss: 1.2991 - policy_loss: 1.1961 - value_loss: 0.1030 - policy_accuracy: 0.7479 - value_accuracy: 0.0480


In [19]:
anet.save_params(8, "7x7")

In [18]:
anet.select_action(HexState.empty_board(), greedy=True, argmax=True)

HexAction(row=3, col=3)